<a href="https://colab.research.google.com/github/mmaruthi/Deep_Learning_EVA4_Phase1/blob/master/S15-A/Depth_First_200K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! nvidia-smi

Sun May 10 14:55:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!git clone https://github.com/anilbhatt1/DenseDepth_V1.git

fatal: destination path 'DenseDepth_V1' already exists and is not an empty directory.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
cd gdrive/My Drive/DepthProject

/content/gdrive/My Drive/DepthProject


In [0]:
!ls -l

total 4
drwx------ 7 root root 4096 May 10 14:21 DenseDepth_V1


In [0]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O DenseDepth_V1/nyu.h5

--2020-05-10 14:55:56--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.89.27
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.89.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘DenseDepth_V1/nyu.h5’

DenseDepth_V1/nyu.h 100%[===================>] 164.89M  11.0MB/s    in 17s     

2020-05-10 14:56:15 (9.50 MB/s) - ‘DenseDepth_V1/nyu.h5’ saved [172897376/172897376]



In [0]:
ls DenseDepth_V1

augment.py      demo_rgb.npy      LICENSE       PyTorch/     utils.py
callbacks.py    DenseDepth.ipynb  loss.py       README.md
data.py         evaluate.py       model.py      Tensorflow/
demo_depth.npy  examples/         nyu.h5        test.py
demo.py         layers.py         __pycache__/  train.py


In [0]:
import os
import glob
import argparse
import matplotlib

# Keras / TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from keras.models import load_model
from matplotlib import pyplot as plt
from DenseDepth_V1.layers import BilinearUpSampling2D
from DenseDepth_V1.utils import predict

MODEL_PATH = 'DenseDepth_V1/nyu.h5'

# Custom object needed for inference and training
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}

print('Loading model...')

# Load model into GPU / CPU
model = load_model(MODEL_PATH, custom_objects=custom_objects, compile=False)

print('\nModel loaded ({0}).'.format(MODEL_PATH))

Using TensorFlow backend.


Loading model...

Model loaded (DenseDepth_V1/nyu.h5).


In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [0]:
from zipfile import ZipFile
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import io
import skimage
from skimage.transform import resize
from itertools import groupby
import cv2
from tqdm.auto import tqdm
from pathlib import Path
from time import time
from datetime import datetime 
import gc
sns.set()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from functools import reduce
def make_batch(input_list, batch_size):
  def reducer(cumulator, item):
    if len(cumulator[-1]) < batch_size:
      cumulator[-1].append(item)
      return cumulator
    else:
      cumulator.append([item])
    return cumulator
  return reduce(reducer, input_list, [[]])

In [0]:
#!unzip -q "/content/gdrive/My Drive/EVA4P1_S15/FG_BG/FG_BG.zip"
FG_BG_zip =  ZipFile("/content/gdrive/My Drive/FG_BG.zip", 'r')

In [0]:
all_files_1 = [info.filename for info in FG_BG_zip.infolist() if not info.is_dir()]

In [0]:
len(all_files_1)

800008

In [0]:
FG_BG_Images = all_files_1[400008:]

In [0]:
len(FG_BG_Images)

400000

In [0]:
def Depth_Creation(frm,to):
    global k, depth_log 
    # Depth Creation loop

    gc.collect()

    images    = []
    file_name = []
    for fg_bg_file in FG_BG_Images[frm:to]:
        imgdata = FG_BG_zip.read(fg_bg_file)
        img = Image.open(io.BytesIO(imgdata))
        img = img.resize((640, 480)) 
        x = np.clip(np.asarray(img, dtype=float) / 255, 0, 1)
        images.append(x)
        fg_bg_name = fg_bg_file.split('/')[-1].split('.')[0]  # We want to take "Img_fg_bg_184131" from 'G_BG/Img_fg_bg_184131.jpg'
        file_name.append(fg_bg_name)
    images = np.stack(images, axis=0)
    output = predict(model, images) 
    outputs = output.copy()

    gc.collect()
     
    # Depth Image saving loop
    for i in range(outputs.shape[0]):         
        op_img = outputs[i]
        op_img = resize(op_img, (200, 200))
        rescaled = op_img[:,:,0]
        rescaled = rescaled - np.min(rescaled)
        rescaled = rescaled / np.max(rescaled)
        depth    = Image.fromarray(rescaled * 255)
        depth    = depth.convert('L')
        depth_name  = file_name[i] + '_depth' + '.jpg'
        depth.save(f'{path_fg_bg_depth}{depth_name}')
        string = f'{depth_name},{file_name[i]}' + '\n'
        depth_log.write(string)
        k += 1

    # cleanup files
    del output, outputs, images

    # garbage collect
    gc.collect()
    return k   


In [0]:
x        = 0
delta    = 1000
iter_lst = []
for i in range(200):
    x1, y1 = x, x+delta
    tup = (x1,y1)
    iter_lst.append(tup)
    x = y1

In [0]:
iter_lst

[(0, 1000),
 (1000, 2000),
 (2000, 3000),
 (3000, 4000),
 (4000, 5000),
 (5000, 6000),
 (6000, 7000),
 (7000, 8000),
 (8000, 9000),
 (9000, 10000),
 (10000, 11000),
 (11000, 12000),
 (12000, 13000),
 (13000, 14000),
 (14000, 15000),
 (15000, 16000),
 (16000, 17000),
 (17000, 18000),
 (18000, 19000),
 (19000, 20000),
 (20000, 21000),
 (21000, 22000),
 (22000, 23000),
 (23000, 24000),
 (24000, 25000),
 (25000, 26000),
 (26000, 27000),
 (27000, 28000),
 (28000, 29000),
 (29000, 30000),
 (30000, 31000),
 (31000, 32000),
 (32000, 33000),
 (33000, 34000),
 (34000, 35000),
 (35000, 36000),
 (36000, 37000),
 (37000, 38000),
 (38000, 39000),
 (39000, 40000),
 (40000, 41000),
 (41000, 42000),
 (42000, 43000),
 (43000, 44000),
 (44000, 45000),
 (45000, 46000),
 (46000, 47000),
 (47000, 48000),
 (48000, 49000),
 (49000, 50000),
 (50000, 51000),
 (51000, 52000),
 (52000, 53000),
 (53000, 54000),
 (54000, 55000),
 (55000, 56000),
 (56000, 57000),
 (57000, 58000),
 (58000, 59000),
 (59000, 60000),
 (

In [0]:
k = 0
path_fg_bg_depth  = '/content/sample_data/ FG_BG_Depth_200K/'

In [0]:
len(iter_lst)

200

In [0]:
for i in range(len(iter_lst)):
    t1=time()
    frm, to           = iter_lst[i]
    depth_log         = open('/content/sample_data/ Log_Depth.txt', "a")
    k = Depth_Creation(frm,to)
    print('Depth Images created so far:',k, datetime.now())
    depth_log.close()
    t2=time()
    print(f'Model took {(t2-t1):.5f} s')


Depth Images created so far: 1000 2020-05-10 15:02:55.462115
Model took 90.35806 s
Depth Images created so far: 2000 2020-05-10 15:04:21.256005
Model took 85.79383 s
Depth Images created so far: 3000 2020-05-10 15:05:46.364272
Model took 85.10811 s
Depth Images created so far: 4000 2020-05-10 15:07:10.904669
Model took 84.54023 s
Depth Images created so far: 5000 2020-05-10 15:08:40.854659
Model took 89.95011 s
Depth Images created so far: 6000 2020-05-10 15:10:10.688636
Model took 89.83390 s
Depth Images created so far: 7000 2020-05-10 15:11:36.642969
Model took 85.95428 s
Depth Images created so far: 8000 2020-05-10 15:13:01.762994
Model took 85.11991 s
Depth Images created so far: 9000 2020-05-10 15:14:28.089132
Model took 86.32618 s
Depth Images created so far: 10000 2020-05-10 15:15:52.916754
Model took 84.82752 s
Depth Images created so far: 11000 2020-05-10 15:17:18.785578
Model took 85.86875 s
Depth Images created so far: 12000 2020-05-10 15:18:44.352125
Model took 85.56656 s
D

In [0]:
!pwd

/content/gdrive/My Drive/DepthProject


In [0]:
os.chdir('/content/gdrive/My Drive/EVA4P1_S15/FG_BG_Depth')

In [0]:
!pwd

/content/gdrive/My Drive/EVA4P1_S15/FG_BG_Depth


In [0]:
zipname_fg_bg_depth = ' FG_BG_Depth_200K'

def zipfolder(foldername, target_dir):            
    zipobj = zipfile.ZipFile(foldername + '.zip', 'w', zipfile.ZIP_DEFLATED)
    rootlen = len(target_dir) + 1
    for base, dirs, files in os.walk(target_dir):
        for file in files:
            fn = os.path.join(base, file)
            zipobj.write(fn, fn[rootlen:])

In [0]:
zipfolder(zipname_fg_bg_depth, '/content/sample_data/')